In [1]:
#(Requires D-wave's Ocean SDK)
import neal
import dimod
import numpy as np
sampler = dimod.HigherOrderComposite(neal.SimulatedAnnealingSampler())

#The Ising cost function in h J of the allZ Hamiltonian from S4 (of the qwc fragment) 
h = {0:0.15542669077992818, 1: 0.1062290449085607, 2: 0.16326768673564326 }
J = {(0,1):0.15660062488237944, (0,2): 0.1062290449085607, (0,3): -0.04919764587136754,
    (1,2):0.15542669077992818, (2,3):0.04919764587136754,
    (0,1,3):0.04919764587136754, (1,2,3):-0.04919764587136754}

offset = -0.3276081896748086 #for the hamiltonian term that didn't have any operators associated with it

sampleset = sampler.sample_hising(h, J, discard_unsatisfied=True)
sample_energy = sampleset.record[0][1]
total_energy = sample_energy  + offset
print("The total energy is: ",total_energy)


The total energy is:  -0.7278419329192338


In [3]:
#Lets try that on an exact solver
sampler = dimod.HigherOrderComposite(dimod.ExactSolver())
sampleset = sampler.sample_hising(h, J, discard_unsatisfied=True)
print("Just add the offset value")
print(sampleset)


Just add the offset value
    0  1  2  3  energy num_oc. penalt.
6  +1 -1 -1 +1 -0.418264       1    True
25 +1 -1 -1 +1 -0.418264       1    True
1  -1 +1 -1 -1 -0.418264       1    True
9  -1 +1 -1 +1 -0.418264       1    True
22 -1 +1 -1 +1 -0.418264       1    True
30 -1 +1 -1 -1 -0.418264       1    True
12 -1 -1 +1 -1 -0.400234       1    True
19 -1 -1 +1 -1 -0.400234       1    True
14 +1 -1 -1 -1 -0.024682       1    True
17 +1 -1 -1 -1 -0.024682       1    True
8  +1 +1 -1 +1 -0.006667       1    True
23 +1 +1 -1 +1 -0.006667       1    True
15 -1 -1 -1 -1 -0.006667       1    True
16 -1 -1 -1 -1 -0.006667       1    True
7  -1 -1 -1 +1 -0.006667       1    True
24 -1 -1 -1 +1 -0.006667       1    True
0  +1 +1 -1 -1 -0.006667       1    True
31 +1 +1 -1 -1 -0.006667       1    True
4  -1 -1 +1 +1 -0.006653       1    True
27 -1 -1 +1 +1 -0.006653       1    True
2  -1 +1 +1 -1 0.006667       1    True
29 -1 +1 +1 -1 0.006667       1    True
5  +1 -1 +1 +1 0.006667       1    

In [7]:
#Now take a 12 variable random problem to see the effects of beta range
np.random.seed(43)
hrand = {}
Jrand = {}
init_st = {}
maxvar = 12
for i in range(0,maxvar):
    if np.random.random() > .5:
        init_st[i] =  1
    else:
        init_st[i] = -1
for i in range(0,maxvar):
    hrand[i] =  (1 + 1) * np.random.random() - 1
for i in range(0,maxvar):
    for j in range(i+1,maxvar):
        if np.random.random() > .95:
            Jrand[(i,j)] = (1 + 1) * np.random.random() - 1
        for k in range(j+1,maxvar):
            if np.random.random() > .95:
                Jrand[(i,j,k)] = (1 + 1) * np.random.random() - 1

#print(init_st)
print("#####")
#print(Jrand)

sampler_rand = dimod.HigherOrderComposite(neal.SimulatedAnnealingSampler())

sampleset_rand = sampler_rand.sample_hising(hrand,Jrand,discard_unsatisfied=False, num_reads=4)
print("Default beta_range")
print(sampleset_rand)
#sampleset = sampler.sample_hising(hrand, Jrand, discard_unsatisfied=True, num_reads=4,initial_state={0:-1,1:-1,2:-1,3:-1},beta_range=[4.2, 4.2])
print("Beta_range 54 to 55, more localized search")
sampleset_rand = sampler_rand.sample_hising(hrand,Jrand,discard_unsatisfied=False, num_reads=4,beta_range=[54, 55])
print(sampleset_rand)

#####
Default beta_range
   0  1  2  3  4  5  6  7  8  9 10 11     energy num_oc. penalt.
0 +1 +1 -1 +1 +1 +1 +1 -1 -1 -1 +1 -1 -12.541803       1       0
1 +1 +1 -1 +1 +1 +1 +1 -1 -1 -1 +1 -1 -12.541803       1       0
2 +1 +1 -1 +1 +1 +1 +1 -1 -1 -1 +1 -1 -12.541803       1       0
3 +1 +1 -1 +1 +1 -1 +1 -1 -1 -1 +1 -1   -9.95032       1       0
['SPIN', 4 rows, 4 samples, 12 variables]
Beta_range 54 to 55, more localized search
   0  1  2  3  4  5  6  7  8  9 10 11   energy num_oc. penalt.
1 +1 +1 -1 +1 +1 -1 +1 -1 -1 -1 +1 -1 -9.95032       1       0
3 +1 +1 -1 +1 +1 -1 +1 -1 -1 -1 +1 -1 -9.95032       1       0
2 -1 +1 -1 +1 +1 +1 +1 -1 -1 -1 +1 +1 -9.539537       1       0
0 +1 +1 -1 +1 +1 -1 +1 -1 -1 -1 +1 +1 -6.03596       1       0
['SPIN', 4 rows, 4 samples, 12 variables]
